In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

# Задание 2

In [84]:
df = pd.concat(pd.read_csv(f'../lab2/20250{i}/players.csv') for i in range(1,5))
df

C:\Users\Roma\AppData\Local\Temp\ipykernel_11260\3887877372.py:1: DtypeWarning: Columns (148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(pd.read_csv(f'../lab2/20250{i}/players.csv') for i in range(1,5))
C:\Users\Roma\AppData\Local\Temp\ipykernel_11260\3887877372.py:1: DtypeWarning: Columns (51,52,128,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(pd.read_csv(f'../lab2/20250{i}/players.csv') for i in range(1,5))


,player_slot,obs_placed,sen_placed,creeps_stacked,camps_stacked,rune_pickups,firstblood_claimed,teamfight_participation,towers_killed,roshans_killed,...,purchase_rapier,match_id,leagueid,performance_others,additional_units,repicked,hero_variant,neutral_tokens_log,neutral_item_history,item_neutral2
0,0,0.0,0.0,4.0,2.0,1.0,0.0,0.863636,0.0,0.0,...,NaN,8107483721,17163,NaN,NaN,NaN,1.0,"[{'time': 433, 'key': 'Tier1Token'}, {'time': ...",NaN,NaN
1,1,2.0,0.0,3.0,1.0,5.0,0.0,0.636364,3.0,0.0,...,NaN,8107483721,17163,NaN,NaN,NaN,1.0,"[{'time': 1918, 'key': 'Tier3Token'}]",NaN,NaN
2,2,0.0,0.0,6.0,2.0,7.0,0.0,0.227273,2.0,1.0,...,NaN,8107483721,17163,NaN,NaN,NaN,1.0,"[{'time': 473, 'key': 'Tier1Token'}, {'time': ...",NaN,NaN
3,3,9.0,12.0,66.0,12.0,1.0,0.0,0.227273,0.0,0.0,...,NaN,8107483721,17163,NaN,NaN,NaN,1.0,[],NaN,NaN
4,4,4.0,5.0,43.0,7.0,3.0,0.0,0.818182,1.0,0.0,...,NaN,8107483721,17163,NaN,NaN,NaN,1.0,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,128,3.0,2.0,8.0,3.0,0.0,0.0,0.400000,0.0,0.0,...,NaN,8245079060,17211,NaN,NaN,NaN,3.0,[],"[{'time': 488, 'item_neutral': 'mana_draught',...",1586.0
6416,129,0.0,0.0,0.0,0.0,3.0,0.0,0.266667,0.0,0.0,...,NaN,8245079060,17211,NaN,NaN,NaN,1.0,[],"[{'time': 450, 'item_neutral': 'polliwog_charm...",1577.0
6417,130,1.0,1.0,3.0,1.0,2.0,0.0,0.333333,1.0,0.0,...,NaN,8245079060,17211,NaN,NaN,NaN,2.0,[],"[{'time': 458, 'item_neutral': 'spark_of_coura...",1594.0
6418,131,2.0,0.0,0.0,0.0,11.0,0.0,0.866667,2.0,0.0,...,NaN,8245079060,17211,NaN,NaN,NaN,1.0,[],"[{'time': 461, 'item_neutral': 'trusty_shovel'...",1583.0


In [12]:
df['gold_t']

0       [0, 287, 377, 607, 1107, 1482, 1865, 2188, 290...
1       [0, 374, 670, 1153, 1608, 2047, 2439, 2996, 34...
2       [0, 325, 657, 1107, 1485, 1929, 2402, 2938, 36...
3       [0, 169, 259, 406, 496, 586, 676, 815, 1097, 1...
4       [0, 169, 296, 437, 576, 666, 756, 846, 1024, 1...
                              ...                        
6415    [0, 170, 260, 350, 487, 625, 789, 1025, 1403, ...
6416    [0, 333, 609, 926, 1334, 1723, 2050, 2663, 304...
6417    [0, 251, 514, 818, 1018, 1456, 1688, 2255, 255...
6418    [0, 205, 604, 1096, 1503, 1956, 2334, 2761, 33...
6419    [0, 170, 331, 577, 691, 798, 888, 1217, 1307, ...
Name: gold_t, Length: 83460, dtype: object

In [ ]:
# Выбор признаков и целевой переменной
features = [
    'gold_per_min', 
    'xp_per_min', 
    'kills', 
    'deaths', 
    'assists',
    'hero_damage', 
    'tower_damage', 
    'last_hits', 
    'denies',
    'duration',
    'lane_role'
]

target = 'radiant_win'
#выбираем только тех игроков, которые за "свет"
df = df[df['player_slot'].isin([0, 1, 2, 3, 4])]
data = df[features + [target]]
data['radiant_win'] = data['radiant_win'].apply(lambda x : 1 if x else 0)
data

,gold_per_min,xp_per_min,kills,deaths,assists,hero_damage,tower_damage,last_hits,denies,duration,lane_role,radiant_win
0,595,793,8,8,11,23790,1082,228,0,2059,3.0,0
1,616,748,6,4,8,19636,4663,286,1,2059,2.0,0
2,764,1061,3,3,2,11558,4904,494,1,2059,1.0,0
3,255,456,1,6,5,8902,0,62,1,2059,1.0,0
4,325,540,4,10,14,16204,373,23,0,2059,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6410,367,521,4,4,20,14897,758,68,3,1723,3.0,1
6411,699,904,13,1,8,21190,4436,255,9,1723,2.0,1
6412,592,614,6,5,19,20022,1248,155,4,1723,3.0,1
6413,569,622,4,2,12,11146,4405,225,5,1723,1.0,1


In [86]:
print(f"Исходный размер данных: {data.shape}")
data = data.dropna()

# Разделение на признаки и целевую переменную
X = data[features]
y = data[target].astype(int)

Исходный размер данных: (41725, 12)


In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42
)

In [88]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("Без нормализации:")
print(classification_report(y_test, y_pred))

Без нормализации:
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      6028
           1       0.75      0.75      0.75      6490

    accuracy                           0.74     12518
   macro avg       0.74      0.74      0.74     12518
weighted avg       0.74      0.74      0.74     12518



In [89]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [90]:
knn_scaled = KNeighborsClassifier(n_neighbors=3)
knn_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = knn_scaled.predict(X_test_scaled)

print("С нормализацией:")
print(classification_report(y_test, y_pred_scaled))

С нормализацией:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      6028
           1       0.89      0.90      0.89      6490

    accuracy                           0.89     12518
   macro avg       0.89      0.89      0.89     12518
weighted avg       0.89      0.89      0.89     12518



Видим, что метрики выросли 15-16%

In [95]:
# Матрица корреляций
plt.figure(figsize=(12,8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.show()
plt.close()

# Выбор признаков с корреляцией > 0.1
selected_features = [
    'gold_per_min', 
    'xp_per_min', 
    'kills', 
    'hero_damage', 
    'tower_damage'
]

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [94]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Важность признаков
pd.Series(model.feature_importances_, index=X.columns).sort_values().plot.barh()
plt.show()

In [96]:
k_values = np.arange(1, 31)
cv_scores = []
train_scores = []
test_scores = []

for k in k_values:
    # Модель
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Кросс-валидация на всем датасете
    cv_score = cross_val_score(knn, X_train_scaled, y_train, cv=5).mean()
    cv_scores.append(cv_score)
    
    # Обучение на тренировочных данных
    knn.fit(X_train_scaled, y_train)
    
    # Оценка точности
    train_scores.append(knn.score(X_train_scaled, y_train))
    test_scores.append(knn.score(X_test_scaled, y_test))

# 6. Построение графиков
plt.figure(figsize=(12, 6))
plt.plot(k_values, train_scores, label='Train Accuracy', marker='o')
plt.plot(k_values, test_scores, label='Test Accuracy', marker='s')
plt.plot(k_values, cv_scores, label='CV Accuracy', marker='^')

plt.title('Зависимость точности от значения K')
plt.xlabel('Значение K')
plt.ylabel('Accuracy')
plt.xticks(k_values)
plt.legend()
plt.grid(True)
plt.show()

# 7. Анализ результатов
best_k_cv = k_values[np.argmax(cv_scores)]
best_k_test = k_values[np.argmax(test_scores)]

print(f'Лучшее K по кросс-валидации: {best_k_cv}')
print(f'Лучшее K по тестовым данным: {best_k_test}')

NameError: name 'cross_val_score' is not defined